this is another messy notebook! this one is used to study jet trscer volume fraction

In [ ]:
import glob
import os
from natsort import natsorted
from postprocessing import *

In [ ]:
mpl.rcParams['figure.dpi']= 400

In [ ]:
density = '20'
mach    = '2'
jet     = '40'
stage   = '25'

In [ ]:
simulation_directory_jet = f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/jet{jet}_{stage}'
output_directory_jet = simulation_directory_jet + "/output/"
figures_directory_jet = simulation_directory_jet + "/output/figures/"

In [ ]:
i_file = 12
filename = "snap_%03d.hdf5" % (i_file)
snap_data = h5py.File(output_directory_jet + filename, "r")

In [ ]:
x = snap_data['PartType0/Jet_Tracer'][:]
masses = snap_data['PartType0/Masses'][:]
energy = snap_data['PartType0/InternalEnergy'][:]
temperature = get_temp(output_directory_jet + filename, 5/3)

In [ ]:
plt.scatter(x, energy, s=2)
plt.axvline(1e-3, c='grey')
plt.xscale('log')
plt.yscale('log')
plt.xlim(1e-13, 10)
plt.ylim(1, 1e7)

plt.xlabel('jet tracer')
plt.ylabel('temperature, K')

In [ ]:
warm_jet = np.zeros_like(x)
mask_warm_jet = (temperature < 2e4) & (x > 1e-3)
warm_jet[mask_warm_jet] = 1

hot_nonjet = np.zeros_like(x)
mask_hot_nonjet = (temperature > 4e4) & (x < 1e-6)
hot_nonjet[mask_hot_nonjet] = 1

In [ ]:
fig, ax = plt.subplots(figsize=(6,7))
# plot_random(ax, output_directory_jet + filename, value=warm_jet, limits=[0, 1], log=False,
#             value_name='warm jet', fac=0.5, t0=15)
plot_random(ax, output_directory_jet + filename, value=hot_nonjet, limits=[0, 1], log=False,
            value_name='hot not jet', fac=0.5, t0=0)
plot_jet_tracer_on_top(ax, output_directory_jet + filename, fac=0.5, t0=0)

In [ ]:
k1 = snap_data['Config'].attrs['BH_CONSTANT_POWER_RADIOMODE_IN_ERG_PER_S']
print(k1)

In [ ]:
sum(x > 0)#snap_data['PartType0/Coordinates']

In [ ]:
plt.hist(np.log10(x[x>1e-25]), bins=100, log=True)
plt.axvline(-3, c='black', zorder=100)
plt.xlabel('jet tracer')

In [ ]:
density_arr  = np.array([30, 60, 100, 200, 300])
jetpower_arr = np.array([41, 42, 43, 44])
machnum_arr  = np.array([4, 8, 12])

In [ ]:
threshold_tracer = 1e-3

In [ ]:
def calculate_jet_tracer_dj(i_file, density_arr=density_arr, jetpower_arr=jetpower_arr, mach=8):
    """
    Create a table with jet tracer volume fraction for different density and jet power values
    Input: i_file       (snapshot number)
           density_arr  (array of number densities for which simulations exist)
           jetpower_arr (array of jet power values for which simulations exist)
           mach         (mach number)
    Output: array with three columns: number density, jet power, and jet volume fraction. and time of the snapshot in Myr
    
    """
    
    result = []
    for density in density_arr:
        for jetpower in jetpower_arr:
            
            simulation_directory_jet = str(f'/n/holyscratch01/hernquist_lab/borodina/turb_drive_center_d{density}_m{mach}/jet{jetpower}_early')
            output_directory_jet = simulation_directory_jet + "/output/"
            filename = "snap_%03d.hdf5" % (i_file)
            try:
                snap_data = h5py.File(output_directory_jet + filename, "r")
                volume = snap_data['PartType0/Masses'][:]/snap_data['PartType0/Density'][:] 
                tracer = snap_data['PartType0/Jet_Tracer'][:]
                jet_volume = volume[tracer > threshold_tracer]
                jet_volume_fraction = np.sum(jet_volume)/np.sum(volume)
                result.append([density, jetpower, jet_volume_fraction])
            except:
                result.append([density, jetpower, -1])
    time = get_time_from_snap(snap_data)
    return np.array(result).T, time * unit_time_in_megay

In [ ]:
xaxis = np.array([1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5])

for i in [0, 1, 2, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]:
    result, time = calculate_jet_tracer_dj(i)
    
    fig, ax = plt.subplots(figsize=(4.5, 3))
    norm=plt.Normalize(np.log10(5e-4), np.log10(0.2))

    mask = (result[2] > 0)

    sc = ax.scatter(xaxis[mask], result[1][mask], c=np.log10(result[2][mask]), 
                    cmap='plasma', norm=norm, marker='s', s=1580, zorder=100)
    ax.tick_params(labelsize=12, direction='in')

    locs, labels = plt.xticks()  # Get the current locations and labels.
    plt.xticks([1, 2, 3, 4, 5], density_arr)  # Set text labels and properties.
    ax.set_xlim(0.5, 5.5)
    ax.set_ylim(40.5, 44.5)

    ax.set_xlabel('density [cm$^{-3}$]', fontsize=15)
    ax.set_ylabel('log (jet power)', fontsize=15)

    cbar = plt.colorbar(sc, format='%g')
    cbar.set_label(label=r'log(jet tracer volume fraction)', size=10)

    ax.set_title("t=%.2f Myr"%(time * unit_time_in_megayr))
    plt.savefig( f'./figures/jet_tracer_dj_{i}.png', bbox_inches='tight', dpi=300)
    plt.close()

In [ ]:
def calculate_jet_tracer_mnum(i_file, machnum_arr=machnum_arr, jetpower_arr=jetpower_arr, density=100):
    """
    Create a table with jet tracer volume fraction for different mach numbers and jet power values
    Input: i_file       (snapshot number)
           machnum_arr  (array of mach numbers for which simulations exist)
           jetpower_arr (array of jet power values for which simulations exist)
           density      (number density of the simulation)
           
    Output: array with three columns: mach number, jet power, and jet volume fraction. and time of the snapshot in Myr
    
    """
    result = []
    for machnum in machnum_arr:
        for jetpower in jetpower_arr:
            simulation_directory_jet = str(f'/n/holyscratch01/hernquist_lab/borodina/turb_drive_center_d{density}_m{machnum}/jet{jetpower}_early')
            output_directory_jet = simulation_directory_jet + "/output/"
            filename = "snap_%03d.hdf5" % (i_file)
            try:
                snap_data = h5py.File(output_directory_jet + filename, "r")
                volume = snap_data['PartType0/Masses'][:]/snap_data['PartType0/Density'][:] 
                tracer = snap_data['PartType0/Jet_Tracer'][:]
                jet_volume = volume[tracer > threshold_tracer]
                jet_volume_fraction = np.sum(jet_volume)/np.sum(volume)
                result.append([machnum, jetpower, jet_volume_fraction])
            except:
                result.append([machnum, jetpower, -1])
    time = get_time_from_snap(snap_data)
    return np.array(result).T, time * unit_time_in_megay

In [ ]:
xaxis = np.array([1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3])
for i in [0, 1, 2, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]:
    result, time = calculate_jet_tracer_mnum(i)

    fig, ax = plt.subplots(figsize=(2.7, 3))
    norm=plt.Normalize(np.log10(5e-4), np.log10(0.2))

    mask = (result[2] > 0)

    sc = ax.scatter(xaxis[mask], result[1][mask], c=np.log10(result[2][mask]), 
                    cmap='plasma', norm=norm, marker='s', s=1580, zorder=100)
    ax.tick_params(labelsize=12, direction='in')

    locs, labels = plt.xticks()  # Get the current locations and labels.
    plt.xticks([1, 2, 3], machnum_arr)  # Set text labels and properties.
    ax.set_xlim(0.5, 3.5)
    ax.set_ylim(40.5, 44.5)

    ax.set_xlabel('Mach number', fontsize=15)
    ax.set_ylabel('log (jet power)', fontsize=15)

    cbar = plt.colorbar(sc, format='%g')
    cbar.set_label(label=r'log(jet tracer volume fraction)', size=10)

    ax.set_title("t=%.2f Myr"%(time * unit_time_in_megayr))
    plt.savefig( f'./figures/jet_tracer_mnum_{i}.png', bbox_inches='tight', dpi=300)
    plt.close()

## make gif

In [ ]:
from PIL import Image

# make gif
#--------------------------
def crop_img(im):
    width, height = im.size
    left = 9
    top =  3
    right = width - 3
    bottom = height - 9
    im = im.crop((left, top, right, bottom))
    return im

ifilename = './figures/jet_tracer_dj*.png'
ofilename = './figures/jet_tracer-dj.gif'
imgs = natsorted(glob.glob(ifilename))

frames = []
for i in imgs:
    new_frame = Image.open(i)
    frames.append(crop_img(new_frame))

frames[0].save(ofilename, format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=140, loop=0)

## time evolution with density

In [ ]:
def jet_and_density(storage, boxsize, density, machnumber, jetpower, start='early'):
    """
    Create a table with jet tracer volume fraction and central density evolution with time
    Input: density    (number density of the simulation)
           machnumber (mach number of the simulation)
           jetpower   (jet power of the simulation)
           start      (initial snapshot from the turbulent box)
           
    Output: array with three columns: time un Myr, jet tracer volume fraction, and mean number density within central 50pc
    
    """    
    simulation_directory_jet = str(f'/n/{storage}/LABS/hernquist_lab/Users/borodina/{boxsize}/turb_jet_d{density}_m{machnumber}/jet{jetpower}_{start}')
    output_directory_jet = simulation_directory_jet + "/output/"
    figures_directory_jet = simulation_directory_jet + "/output/figures/"
    
    threshold_tracer = 1e-3
    result = []
    i_file = 12
    while True: #True or a number for a set length
        i_file += 1
        output_directory_jet = simulation_directory_jet + "/output/"
        filename = "snap_%03d.hdf5" % (i_file)
        try:
            snap_data = h5py.File(output_directory_jet + filename, "r+")
        except:
            try:
                i_file += 1
                filename = "snap_%03d.hdf5" % (i_file)
                snap_data = h5py.File(output_directory_jet + filename, "r+")
            except:
                break
        time = get_time_from_snap(snap_data) * unit_time_in_megayr
        x, y, z = snap_data['PartType0/Coordinates'][:].T
        center = snap_data['Header'].attrs['BoxSize'] / 2
        mask = (np.sqrt((x - center) ** 2 + (y - center) ** 2 + (z - center) ** 2) < 500)
        
        volume = snap_data['PartType0/Masses'][:][mask]/snap_data['PartType0/Density'][:][mask]
        tracer = snap_data['PartType0/Jet_Tracer'][:][mask]
        jet_volume = volume[tracer > threshold_tracer]
        jet_volume_fraction = np.sum(jet_volume)/np.sum(volume)
        mask = (np.sqrt((x - center) ** 2 + (y - center) ** 2 + (z - center) ** 2) < 50)
        n_mean = np.mean(snap_data['PartType0/Density'][mask]) * rho_to_numdensity
        result.append([time, jet_volume_fraction, n_mean])
    return result

In [ ]:

result_8  = np.array(jet_and_density('holystore01', '2kpc', 30,  8, 43, 8)).T
result_12  = np.array(jet_and_density('holystore01', '2kpc', 30,  8, 43, 12)).T
result_15 = np.array(jet_and_density('holystore01', '2kpc', 30, 8, 43, 15)).T
result_early = np.array(jet_and_density('holystore01', '1kpc',30,  8, 43, 'early')).T
result_uni = np.array(jet_and_density('holylfs05', '2kpc',30, 8, 43, 'uniform')).T


In [ ]:
result_mean = np.mean([result_8[1][:24], result_12[1][:24], result_15[1][:24]], axis=0)
result_min = np.min([result_8[1][:24], result_12[1][:24], result_15[1][:24]], axis=0)
result_max = np.max([result_8[1][:24], result_12[1][:24], result_15[1][:24]], axis=0)

In [ ]:
fig, ax1 = plt.subplots(figsize=(5,3))


# ax1.plot(result_8[0], result_8[1], label='m8 jet=43', c='darkred', ls='--', )
# ax1.plot(result_12[0], result_12[1], c='darkred', ls='--', )
# ax1.plot(result_15[0], result_15[1], c='darkred', ls='--', )
ax1.plot(result_uni[0], result_uni[1], label='jet in uniform box', c='black', ls='-')
#ax1.plot(result_early[0], result_early[1], label='1 kpc box', c='darkblue', ls='--')

ax1.plot(result_8[0], result_mean, '-', c='darkred', label='jet in turbulent ISM')
ax1.fill_between(result_8[0], result_min, result_max, alpha=0.2, color='darkblue', zorder=100)

# ax1.axhline(0.05, c='gray', ls='--')

ax1.set_xlabel('t [Myr]', fontsize=10)
ax1.set_xlim(0, 1.02)
ax1.set_ylim(1e-3, 0.5)
# ax2.set_ylim(10, 6e3)

# ax1.set_facecolor('w')
# fig.set_facecolor('none')

params = {"ytick.color" : "w",
      "xtick.color" : "w",
      "axes.labelcolor" : "w",
      "axes.edgecolor" : "w",
      "text.color" : "w",
      "figure.facecolor" : '2F366E'}
plt.rcParams.update(params)

ax1.set_ylabel('jet tracer volume fraction', fontsize=10, zorder=100)
plt.title(r'$n = 30$cm$^{-3}$  $\quad \mathfrak{M} = 8 \quad$ jet power$= 10^{43}$erg/s')
plt.grid(ls='--', c='gray', alpha=0.4, zorder=0)
plt.legend(fontsize=9, loc='upper left', labelcolor='black')
plt.show()

In [ ]:
result_43  = np.array(jet_and_density('holylfs05', '2kpc', 10, 8, 43, 15)).T
result_40  = np.array(jet_and_density('holylfs05', '2kpc', 10, 8, 40, 15)).T
result_38  = np.array(jet_and_density('holylfs05', '2kpc', 10, 8, 38, 15)).T

In [ ]:
result_43_uniform  = np.array(jet_and_density('holylfs05', '2kpc', 10, 8, 43, 'uniform')).T
result_40_uniform  = np.array(jet_and_density('holylfs05', '2kpc', 10, 8, 40, 'uniform')).T
result_38_uniform  = np.array(jet_and_density('holylfs05', '2kpc', 10, 8, 38, 'uniform')).T

In [ ]:
result_43_no_ss  = np.array(jet_and_density('holylfs05', '2kpc', 10, 8, 43, 'no_ss')).T
result_40_no_ss  = np.array(jet_and_density('holylfs05', '2kpc', 10, 8, 40, 'no_ss')).T
result_38_no_ss  = np.array(jet_and_density('holylfs05', '2kpc', 10, 8, 38, 'no_ss')).T

In [ ]:
result_40_12  = np.array(jet_and_density('holylfs05', '2kpc', 20, 12, 40, 15)).T
result_40_8  = np.array(jet_and_density('holylfs05', '2kpc', 20, 8, 40, 15)).T
result_40_4  = np.array(jet_and_density('holylfs05', '2kpc', 20, 4, 40, 15)).T

In [ ]:
result_40_uniform  = np.array(jet_and_density('holylfs05', '2kpc', 20, 8, 40, 'uniform')).T
result_40_uniform_dilute  = np.array(jet_and_density('holylfs05', '2kpc', 20, 8, 40, 'uniform_dilute')).T

In [ ]:
fig, ax1 = plt.subplots(figsize=(5,3))

# ax1.plot(result_43[0], result_43[1], label='jet 10^43 erg/s', c='darkred', ls='-', )
# ax1.plot(result_40[0], result_40[1], label='jet 10^40 erg/s', c='tab:blue', ls='-', )
# ax1.plot(result_38[0], result_38[1], label='jet 10^38 erg/s', c='tab:green', ls='-', )

# ax1.plot(result_43_uniform[0], result_43_uniform[1], label='jet 10^43 erg/s uniform', c='darkred', ls='--', )
# ax1.plot(result_40_uniform[0], result_40_uniform[1], label='jet 10^40 erg/s uniform', c='tab:blue', ls='--', )
# ax1.plot(result_38_uniform[0], result_38_uniform[1], label='jet 10^38 erg/s uniform', c='tab:green', ls='--', )

# ax1.plot(result_43_no_ss[0], result_43_no_ss[1], label='jet 10^43 erg/s v=0', c='darkred', ls=':', )
# ax1.plot(result_40_no_ss[0], result_40_no_ss[1], label='jet 10^40 erg/s v=0', c='tab:blue', ls=':', )
# ax1.plot(result_38_no_ss[0], result_38_no_ss[1], label='jet 10^38 erg/s v=0', c='tab:green', ls=':', )


ax1.plot(result_40_12[0], result_40_12[1], label='M = 12', c='purple', ls='-', )
ax1.plot(result_40_8[0], result_40_8[1], label='M = 8', c='tab:blue', ls='-', )
ax1.plot(result_38_4[0], result_38_4[1], label='M = 4', c='tab:green', ls='-')

ax1.plot(result_40_uniform[0], result_40_uniform[1], label='uniform', c='navy', ls='-', )
ax1.plot(result_40_uniform_dilute[0], result_40_uniform_dilute[1], label='uniform dilute', c='aqua', ls='-', )

# ax1.axhline(0.05, c='gray', ls='--')

ax1.set_xlabel('t [Myr]', fontsize=10)
ax1.set_xlim(15, 21)
ax1.set_ylim(0, 0.4)
# ax2.set_ylim(10, 6e3)

ax1.set_ylabel('jet tracer volume fraction', fontsize=10, zorder=100)
# ax2.set_ylabel('mean density inside central 50 pc [cm$^{-3}$](-)', fontsize=10)
# ax2.set_yscale('log')
# ax1.set_yscale('log')
plt.grid(ls='--', c='gray', alpha=0.4, zorder=0)
plt.legend(fontsize=6, loc='upper right')
plt.show()

## jet tracer radial percentiles

In [ ]:
def percentile_distance(storage, boxsize, density, machnumber, jetpower, start='early', percentiles=[10, 50, 100]):
    """
    Create a table with jet tracer volume fraction and central density evolution with time
    Input: density    (number density of the simulation)
           machnumber (mach number of the simulation)
           jetpower   (jet power of the simulation)
           start      (initial snapshot from the turbulent box)
           a
    Output: array with three columns: time un Myr, jet tracer volume fraction, and mean number density within central 50pc
    
    """    
    simulation_directory = str(f'/n/{storage}/LABS/hernquist_lab/Users/borodina/{boxsize}/turb_jet_d{density}_m{machnumber}/jet{jetpower}_{start}')
    output_directory = simulation_directory + "/output/"
    figures_directory = simulation_directory + "/output/figures/"
    
    threshold_tracer = 1e-3
    i_file = 12
    distance_array = []
    time = 0
    while True and time < 20: #True or a number for a set length
        i_file += 1
        filename = "snap_%03d.hdf5" % (i_file)
        try:
            snap_data = h5py.File(output_directory + filename, "r+")
        except:
            try:
                i_file += 1
                filename = "snap_%03d.hdf5" % (i_file)
                snap_data = h5py.File(output_directory + filename, "r+")
            except:
                break
        center = 1000
        x = snap_data['PartType0/Coordinates'][:, 0] - center
        y = snap_data['PartType0/Coordinates'][:, 1] - center
        z = snap_data['PartType0/Coordinates'][:, 2] - center
        jet_tracer = snap_data['PartType0/Jet_Tracer'][:]
        mass = snap_data['PartType0/Masses'][:]

        mask_jet = jet_tracer > 1e-3
        time = get_time_from_snap(snap_data) * unit_time_in_megayr
        distance_sq = x ** 2 + y ** 2 + z ** 2

        if len(distance_sq[mask_jet]) > 0:
            #distance_max = np.sqrt(distance_sq[mask_jet].max())
            distances = np.sqrt(np.percentile(distance_sq[mask_jet], percentiles))
        else:
            distances = np.zeros_like(percentiles)

        distance_array.append([np.ones_like(distances) * time, distances])
    return np.array(distance_array)

In [ ]:
percentiles = np.linspace(5, 100, 51)
# dist_38_15 = percentile_distance('holylfs05', '2kpc', 10, 8, 38, '15', percentiles)
# dist_40_15 = percentile_distance('holylfs05', '2kpc', 10, 8, 40, '15', percentiles)
# dist_43_15 = percentile_distance('holylfs05', '2kpc', 10, 8, 43, '15', percentiles)

dist_38_uniform = percentile_distance('holylfs05', '2kpc', 10, 8, 38, 'uniform', percentiles)
dist_40_uniform = percentile_distance('holylfs05', '2kpc', 10, 8, 40, 'uniform', percentiles)
dist_43_uniform = percentile_distance('holylfs05', '2kpc', 10, 8, 43, 'uniform', percentiles)

In [ ]:
for i in range(len(dist_40_uniform)):
    plt.scatter(dist_40_uniform[i][0],dist_40_uniform[i][1], c=percentiles, cmap='plasma', s=10)
plt.colorbar(label='percentile')
plt.xlim(15,20)
plt.ylim(0, 500 * np.sqrt(2))
plt.xlabel('time [Myr]')
plt.ylabel('distance [pc]')

In [ ]:
dist_40_15[1][0]

In [ ]:
def weighted_percentile(data, weights, perc):
    """
    perc : percentile in [0-1]!
    """
    ix = np.argsort(data)
    data = data[ix] # sort data
    weights = weights[ix] # sort weights
    cdf = (np.cumsum(weights) - 0.5 * weights) / np.sum(weights) # 'like' a CDF function
    return np.interp(perc, cdf, data)

In [ ]:
density = 10
mach = 8
jetpower = 40
start = '15'

simulation_directory = str(f'/n/holylfs05/LABS/hernquist_lab/Users/borodina/2kpc/turb_jet_d{density}_m{mach}/jet{jetpower}_{start}')
output_directory = simulation_directory + "/output/"
figures_directory = simulation_directory + "/output/figures/"

i_file = 20
filename = "snap_%03d.hdf5" % (i_file)

snap_data = h5py.File(output_directory + filename, "r+")

center = 1000
x = snap_data['PartType0/Coordinates'][:, 0] - center
y = snap_data['PartType0/Coordinates'][:, 1] - center
z = snap_data['PartType0/Coordinates'][:, 2] - center
mass = snap_data['PartType0/Masses'][:]
jet_tracer = snap_data['PartType0/Jet_Tracer'][:]

mask_jet = jet_tracer > 1e-3
time = get_time_from_snap(snap_data) * unit_time_in_megayr
distance_sq = x ** 2 + y ** 2 + z ** 2

new, old, new_weighted = [], [], []
for i in np.linspace(0, 1, 11):
    new_weighted.append(weighted_percentile(distance_sq[mask_jet], jet_tracer[mask_jet] * mass[mask_jet], i))
    new.append(weighted_percentile(distance_sq[mask_jet], np.ones_like(distance_sq[mask_jet]), i))
    old.append(np.percentile(distance_sq[mask_jet], i * 100))

fig, ax = plt.subplots(figsize=(4, 4))
#plt.scatter(np.sqrt(old), np.sqrt(new), c=-np.linspace(0, 1, 11))
plt.plot([0,150], [0,150], c='black')
plt.scatter(np.sqrt(old), np.sqrt(new_weighted), c=-np.linspace(0, 1, 11), zorder=100)
ax.set_aspect('1')
ax.set_xlabel('distance using basic percentile')
ax.set_ylabel('distance using weigthed percentile')

In [ ]:
def weighted_percentile(data, weights, perc):
    """
    perc : percentile in [0-1]!
    """
    ix = np.argsort(data)
    data = data[ix] # sort data
    weights = weights[ix] # sort weights
    cdf = (np.cumsum(weights) - 0.5 * weights) / np.sum(weights) # 'like' a CDF function
    return np.interp(perc, cdf, data)

In [ ]:
def percentile_distance_3(storage, boxsize, density, machnumber, jetpower, start='early'):
    """
    Create a table with jet tracer volume fraction and central density evolution with time
    Input: density    (number density of the simulation)
           machnumber (mach number of the simulation)
           jetpower   (jet power of the simulation)
           start      (initial snapshot from the turbulent box)
           a
    Output: array with three columns: time un Myr, jet tracer volume fraction, and mean number density within central 50pc
    
    """    
    simulation_directory = str(f'/n/{storage}/LABS/hernquist_lab/Users/borodina/{boxsize}/turb_jet_d{density}_m{machnumber}/jet{jetpower}_{start}')
    output_directory = simulation_directory + "/output/"
    figures_directory = simulation_directory + "/output/figures/"
    
    threshold_tracer = 1e-3
    i_file = 12
    distance_array = []
    time = 0
    while True and time < 20: #True or a number for a set length
        i_file += 1
        filename = "snap_%03d.hdf5" % (i_file)
        try:
            snap_data = h5py.File(output_directory + filename, "r+")
        except:
            try:
                i_file += 1
                filename = "snap_%03d.hdf5" % (i_file)
                snap_data = h5py.File(output_directory + filename, "r+")
            except:
                break
        center = 1000
        x = snap_data['PartType0/Coordinates'][:, 0] - center
        y = snap_data['PartType0/Coordinates'][:, 1] - center
        z = snap_data['PartType0/Coordinates'][:, 2] - center
        jet_tracer = snap_data['PartType0/Jet_Tracer'][:]
        mass = snap_data['PartType0/Masses'][:]

        mask_jet = jet_tracer > 1e-3
        time = get_time_from_snap(snap_data) * unit_time_in_megayr
        distance_sq = x ** 2 + y ** 2 + z ** 2

        if len(distance_sq[mask_jet]) > 0:
#             distance_100 = np.sqrt(np.percentile(distance_sq[mask_jet], 100))
#             distance_80 = np.sqrt(np.percentile(distance_sq[mask_jet], 80))
#             distance_50 = np.sqrt(np.percentile(distance_sq[mask_jet], 50))
            
            distance_100 = np.sqrt(weighted_percentile(distance_sq[mask_jet], jet_tracer[mask_jet] * mass[mask_jet], 1))
            distance_80 = np.sqrt(weighted_percentile(distance_sq[mask_jet], jet_tracer[mask_jet] * mass[mask_jet], 0.8))
            distance_50 = np.sqrt(weighted_percentile(distance_sq[mask_jet], jet_tracer[mask_jet] * mass[mask_jet], 0.5))
        else:
            distance_100 = 0
            distance_80 = 0
            distance_50 = 0

        distance_array.append([time, distance_100, distance_80, distance_50])
    return np.array(distance_array).T

In [ ]:
dist_38_8_15 = percentile_distance_3('holylfs05', '2kpc', 20, 8, 38, '15')
dist_40_8_15 = percentile_distance_3('holylfs05', '2kpc', 20, 8, 40, '15')
dist_43_8_15 = percentile_distance_3('holylfs05', '2kpc', 20, 8, 43, '15')

In [ ]:
dist_40_12_15 = percentile_distance_3('holylfs05', '2kpc', 20, 12, 40, '15')
dist_40_4_15 = percentile_distance_3('holylfs05', '2kpc', 20, 4, 40, '15')

In [ ]:
dist_40_uniform  = percentile_distance_3('holylfs05', '2kpc', 20, 8, 40, 'uniform')

In [ ]:
dist_40_uniform_dilute  = percentile_distance_3('holylfs05', '2kpc', 20, 8, 40, 'uniform_dilute')

In [ ]:
fig, axes = plt.subplots(figsize=(6,4))

# axes.plot(dist_40_4_15[0], dist_40_4_15[2], label='M = 4', c='purple', ls='-', )
axes.plot(dist_40_8_15[0], dist_40_8_15[2], label='M = 8', c='tab:blue', ls='-', )
# axes.plot(dist_40_12_15[0], dist_40_12_15[2], label='M=12', c='tab:green', ls='-', )

# axes.fill_between(dist_40_4_15[0], dist_40_4_15[1], dist_40_4_15[3], color='purple', ls='-', alpha=0.2)
axes.fill_between(dist_40_8_15[0], dist_40_8_15[1], dist_40_8_15[3], color='tab:blue', ls='-', alpha=0.2)
# axes.fill_between(dist_40_12_15[0], dist_40_12_15[1], dist_40_12_15[3], color='tab:green', ls='-', alpha=0.2)

axes.plot(dist_40_uniform[0], dist_40_uniform[1], label='uniform', c='navy', ls='--', )
axes.plot(dist_40_uniform_dilute[0], dist_40_uniform_dilute[1], label='uniform dilute', c='aqua', ls='--', )

axes.fill_between(dist_40_uniform[0], dist_40_uniform[1], dist_40_uniform[3], color='navy', ls='-', alpha=0.2, hatch='///')
axes.fill_between(dist_40_uniform_dilute[0], dist_40_uniform_dilute[1], dist_40_uniform_dilute[3], color='aqua', ls='-', alpha=0.2, hatch='///')

axes.set_xlabel('t [Myr]', fontsize=10)
axes.set_xlim(15, 20)

axes.set_ylabel('jet tracer distance 80th percentile', fontsize=10, zorder=100)
axes.grid(ls='--', c='gray', alpha=0.4, zorder=0)
axes.legend(fontsize=10, loc='upper left', framealpha=0.6)
fig.tight_layout()

In [ ]:
dist_43_uniform  = percentile_distance('holylfs05', '2kpc', 20, 8, 43, 'uniform')
dist_40_uniform  = percentile_distance('holylfs05', '2kpc', 20, 8, 40, 'uniform')
dist_38_uniform  = percentile_distance('holylfs05', '2kpc', 20, 8, 38, 'uniform')

In [ ]:
dist_43_uniform_dilute  = percentile_distance_3('holylfs05', '2kpc', 20, 8, 43, 'uniform_dilute')
dist_40_uniform_dilute  = percentile_distance_3('holylfs05', '2kpc', 20, 8, 40, 'uniform_dilute')
dist_38_uniform_dilute  = percentile_distance_3('holylfs05', '2kpc', 20, 8, 38, 'uniform_dilute')